<a href="https://colab.research.google.com/github/ML4SCI/DeepLearnHackathon/blob/main/ParticleImagesChallenge/ParticleImages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Introduction**

Machine Learning algorithms have become an increasingly important tool for analyzing the data from the Large Hadron Collider (LHC). Identification of particles in LHC collisions is an important task of LHC detector reconstruction algorithms.

Here we present a challenge where one of the detectors (the Electromagnetic Calorimeter or ECAL) is used as a camera to analyze detector images from two types of particles: electrons and photons that deposit their energy in this detector.

**Dataset**

Each pixel in the image corresponds to a detector cell, while the intensity of the pixel corresponds to how much energy is measured in that cell. Timing of the energy deposits are also available, though this may or may not be relevant. The dataset contains 32x32 Images of the energy hits and their timing (channel 1: hit energy and channel 2: its timing) in each calorimeter cell (one cell = one pixel) for the two classes of particles: Electrons and Photons. The dataset contains around four hundred thousand images for electrons and photons. Please note that your final model will be evaluated on an unseen test dataset.

**Algorithm**

Please use a Machine Learning model of your choice to achieve the highest possible classification performance on the provided dataset. Please provide a Jupyter Notebook that shows your solution.

Evaluation Metrics
ROC curve (Receiver Operating Characteristic curve) and AUC score (Area Under the ROC Curve)
Training and Validation Accuracy
The model performance will be tested on a hidden test dataset based on the above metrics.

**Deliverables**

Fill out the pre- and post-hackathon surveys.
Pdf and .ipynb file showing your solution along with the final model accuracy (Training and Validation), ROC curve and AUC score. More details regarding the format of the notebook can be found in the sample Google Colab notebook provided for this challenge.
The final trained model including the model architecture and the trained weights ( For example: HDF5 file, .pb file, .pt file, etc. ). You are free to choose Machine Learning Framework of your choice.


## Create the appropriate project folder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/

In [ ]:
mkdir Particle_Images

In [ ]:
cd Particle_Images

In [ ]:
mkdir data/

# Download the Dataset
This will download 83MB for SingleElectron and 76MB for SinglePhoton.

In [ ]:
#!/bin/bash
!wget https://cernbox.cern.ch/index.php/s/sHjzCNFTFxutYCj/download -O data/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5
!wget https://cernbox.cern.ch/index.php/s/69nGEZjOy3xGxBq/download -O data/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5

# Import modules

In [ ]:
import numpy as np
np.random.seed(42)
import torch
torch.manual_seed(42)
torch.cuda.manual_seed_all
import h5py
from torch import nn
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import roc_curve, auc

import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Pytorch Model Parameters

In [ ]:
lr_init     = 1e-4    # Initial learning rate
batch_size  = 64      # Training batch size
num_epochs = 10       # Number of training epochs

It is recommended to use GPU for training and inference if possible.

# Load Image Data
- Two classes of particles: electrons and photons
- 32x32 matrices (two channels - hit energy and time) for the two classes of particles electrons and photons impinging on a calorimeter (one calorimetric cell = one pixel).
- Note that although timing channel is provided, it may not necessarily help the performance of the model.

In [ ]:
filename = "./data/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5"
data1 = h5py.File(filename, "r")
Y1 = data1["y"]
X1 = data1["X"]
filename = "./data/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5"
data0 = h5py.File(filename, "r")
Y0 = data0["y"]
X0 = data0["X"]
X_final = np.concatenate((X0[:], X1[:]), axis=0)
X_final = np.moveaxis(X_final, 3, 1)
Y_final = np.concatenate((Y0[:], Y1[:]), axis=0)

# Configure Training / Validation / Test Sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_final, #Can change to X_final[:, 0:1, :, :] to use the Hit-Energy channel only
    Y_final,
    test_size=0.2,
    random_state=42
)

X_valid, X_test, y_valid, y_test = train_test_split(
    X_test,
    y_test,
    test_size=0.5,
    random_state=42
)

print(f"X_train shape: {X_train.shape} - y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape} - y_test shape: {y_test.shape}")
print(f"X_valid shape: {X_valid.shape} - y_valid shape: {y_valid.shape}")

# Plot sample of training images
Note that although the timing channel is provided, it may not necessarily help the performance of the model.

In [ ]:
plt.figure(1)

plt.subplot(221)
plt.imshow(X_train[1,0,:,:])
plt.title("Channel 0: Energy")  # Energy
plt.grid(True)

plt.subplot(222)
plt.imshow(X_train[1,1,:,:])
plt.title("Channel 1: Time")  # Time
plt.grid(True)


plt.show()

#Construct Dataloaders



In [ ]:
class ClassificationDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.copy()).float()
        self.y = torch.from_numpy(y.copy()).long()
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_data = ClassificationDataset(X_train, y_train)
valid_data = ClassificationDataset(X_valid, y_valid)
test_data = ClassificationDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)

In [ ]:
del X_train, y_train, X_valid, y_valid, X_test, y_test

# Define CNN Model
This is a sample model. You can experiment with the model and try various architectures and other models to achieve the highest possible performance.  

In [ ]:
### Define Convolutional Neural Network (CNN) Model ###

model = nn.Sequential()
model.append(nn.Conv2d(in_channels=2, out_channels=16, kernel_size=3, padding='same'))
model.append(nn.ReLU())
model.append(nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding='same'))
model.append(nn.ReLU())
model.append(nn. MaxPool2d(kernel_size=(2, 2)))
model.append(nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding='same'))
model.append(nn.ReLU())
model.append(nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding='same'))
model.append(nn.ReLU())
model.append(nn. MaxPool2d(kernel_size=(2, 2)))
model.append(nn.Flatten())
model.append(nn.Linear(1024, 256))
model.append(nn.ReLU())
model.append(nn.Dropout(0.2))
model.append(nn.Linear(256, 128))
model.append(nn.ReLU())
model.append(nn.Dropout(0.2))
model.append(nn.Linear(128, 1))
model.append(nn.Sigmoid())
model.append(nn.Flatten(start_dim=0))
print(model)

## Train the Model
You may further optimize the model, tune hyper-parameters, etc. accordingly to achieve the best performance possible.

In [ ]:
def train_and_validate(train_loader, val_loader, model, optimizer, criterion, num_epochs, metric=None, scheduler=None, device='cpu'):
    history = {
        'epoch': [],
        'train_loss': [],
        'train_metric': [],
        'val_loss': [],
        'val_metric': [],
        'learning_rate': []
    }  # Initialize a dictionary to store epoch-wise results

    model.to(device)  # Move the model to the specified device

    with torch.no_grad():
        proper_dtype = torch.int64
        X,y = next(iter(train_loader))
        X = X.to(device)
        y = y.to(device)
        try:
            loss = criterion(model(X), y.to(proper_dtype))
        except:
            try:
                proper_dtype = torch.float32
                loss = criterion(model(X), y.to(proper_dtype))
            except:
                print("No valid data-type could be found")

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        epoch_loss = 0.0  # Initialize the epoch loss and metric values
        epoch_metric = 0.0

        # Training loop
        for X, y in train_loader:
            X = X.to(device)
            y = y.to(device)
            y = y.to(proper_dtype)
            optimizer.zero_grad()  # Clear existing gradients
            outputs = model(X)  # Make predictions
            loss = criterion(outputs, y)  # Compute the loss
            loss.backward()  # Compute gradients
            optimizer.step()  # Update model parameters

            epoch_loss += loss.item()

            # THESE LINES HAVE BEEN UPDATED TO ACCOUNT FOR DEFAULT ARGUMENTS
            if metric is not None:
                epoch_metric += metric(outputs, y)
            else:
                epoch_metric += 0.0

        # Average training loss and metric
        epoch_loss /= len(train_loader)
        epoch_metric /= len(train_loader)

        # Validation loop
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # Disable gradient calculation
            val_loss = 0.0
            val_metric = 0.0
            for X_val, y_val in val_loader:
                X_val = X_val.to(device)
                y_val = y_val.to(device)
                y_val = y_val.to(proper_dtype)
                outputs_val = model(X_val)  # Make predictions
                val_loss += criterion(outputs_val, y_val).item()  # Compute loss
                if metric is not None:
                    val_metric += metric(outputs_val, y_val)
                else:
                    val_metric += 0.0

            val_loss /= len(val_loader)
            val_metric /= len(val_loader)

        # Append epoch results to history
        history['epoch'].append(epoch)
        history['train_loss'].append(epoch_loss)
        history['train_metric'].append(epoch_metric)
        history['val_loss'].append(val_loss)
        history['val_metric'].append(val_metric)
        history['learning_rate'].append(optimizer.param_groups[0]['lr'])

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, '
              f'Train Metric: {epoch_metric:.4f}, Val Loss: {val_loss:.4f}, '
              f'Val Metric: {val_metric:.4f}')

        if scheduler is not None:
            scheduler.step(val_loss)

    return history, model

In [ ]:
def test_model(model, data_loader, criterion, metric=None, device='cpu'):
    model.to(device)  # Move the model to the specified device

    model.eval()  # Set the model to evaluation mode

    total_loss = 0.0  # Initialize the total loss and metric values
    total_metric = 0.0

    with torch.no_grad():
        proper_dtype = torch.int64
        X,y = next(iter(data_loader))
        X = X.to(device)
        y = y.to(device)
        try:
            loss = criterion(model(X), y.to(proper_dtype))
        except:
            try:
                proper_dtype = torch.float32
                loss = criterion(model(X), y.to(proper_dtype))
            except:
                print("No valid data-type could be found")


    with torch.no_grad():  # Disable gradient tracking
        for batch in data_loader:
            X, y = batch
            X = X.to(device)
            y = y.to(device)
            y = y.to(proper_dtype)
            # Pass the data to the model and make predictions
            outputs = model(X)

            # Compute the loss
            loss = criterion(outputs, y)

            # Add the loss and metric for the batch to the total values
            total_loss += loss.item()

            if metric is not None:
                total_metric += metric(outputs, y)
            else:
                total_metric += 0.0

    # Average loss and metric for the entire dataset
    avg_loss = total_loss / len(data_loader)
    avg_metric = total_metric / len(data_loader)

    print(f'Test Loss: {avg_loss:.4f}, Test Metric: {avg_metric:.4f}')

    return avg_loss, avg_metric

In [ ]:
def accuracy_metric(pred, target):
    if len(pred.shape) == 1:
        accuracy = torch.sum(torch.eq(pred > 0.5, target)).item() / len(pred)
    else:
        pred = pred.argmax(dim=1)
        accuracy = torch.sum(pred == target).item() / len(pred)
    return accuracy

**Note: This may take several hours without GPU**

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr_init)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.2, patience=2)
criterion = nn.BCELoss()
history, model = train_and_validate(train_loader, valid_loader, model, optimizer, criterion, num_epochs=num_epochs, metric=accuracy_metric, scheduler=scheduler, device=device)

## Evaluate the Model  
Along with the model accuracy, the prefered metric for evaluation is ROC (Receiver Operating Characteristic) curve and the AUC score (Area under the ROC Curve).

In [ ]:
# Evaluate on validation set
score = test_model(model, valid_loader, criterion, metric=accuracy_metric, device=device)
print('\nValidation loss / accuracy: %0.4f / %0.4f'%(score[0], score[1]))
model.to(device)
model.eval()
with torch.no_grad():
  y_pred = [model(x[0].to(device)).to('cpu').detach().numpy() for x in valid_loader]
labels = [x[1] for x in valid_loader]
y_pred = np.concatenate(y_pred, axis=0)
labels = np.concatenate(labels, axis=0)
fpr, tpr, _ = roc_curve(labels, y_pred)
roc_auc = auc(fpr, tpr)
print('Validation ROC AUC:', roc_auc)

# Evaluate on test set
score = test_model(model, test_loader, criterion, metric=accuracy_metric, device=device)
print('\nTest loss / accuracy: %0.4f / %0.4f'%(score[0], score[1]))
model.to(device)
model.eval()
with torch.no_grad():
  y_pred = [model(x[0].to(device)).to('cpu').detach().numpy() for x in test_loader]
labels = [x[1] for x in test_loader]
y_pred = np.concatenate(y_pred, axis=0)
labels = np.concatenate(labels, axis=0)
fpr, tpr, _ = roc_curve(labels, y_pred)
roc_auc = auc(fpr, tpr)
print('Test ROC AUC:', roc_auc)

In [ ]:
plt.plot([0, 1], [0, 1], 'k--')
#plt.legend(loc=2, prop={'size': 15})
plt.plot(fpr, tpr, label='Model 1 (ROC-AUC = {:.3f})'.format(roc_auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

# Submission format:
Submit the Google Colab Jupyter Notebook demonstrating your solution in the similar format as illustrated in this notebook. It should contain:
*   The final model architecture, parameters and hyper-parameters yielding the best possible performance,
*   Its Training and Validation accuracy,
*   ROC curve and the AUC score as shown above.
*   Also, please submit the final trained model containing the model architecture and its trained weights along with this notebook (For example: HDF5 file, .pb file, .pt file, etc.). Either in this notebook or in a separate notebook show how to load and use your model.